In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import models
import tensorflow as tf
import pandas as pd 
import numpy as np
import kagglehub
import torch 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Test

In [2]:
# "cuda:0" if torch.cuda.is_available() else 
device =  "cpu"
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-cased")
embed_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-large-cased").to(device)

C:\Users\a\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, pleas

In [3]:
tokens = tokenizer(["this is a test", "test2"], padding=True, return_tensors='pt').to(device)
tokens

{'input_ids': tensor([[ 101, 1142, 1110,  170, 2774,  102],
        [ 101, 2774, 1477,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]])}

In [4]:
embedding = embed_model(**tokens, output_hidden_states=True)
last_hidden = embedding['hidden_states'][-1].cpu().detach().numpy()
last_hidden.reshape(last_hidden.shape[0], last_hidden.shape[1] * last_hidden.shape[2]).shape

(2, 6144)

In [5]:
def get_last_hidden_state(l_of_text):
    tokens = tokenizer(l_of_text, padding="max_length", max_length=60, truncation=True, return_tensors='pt')
    embedding = embed_model(**tokens, output_hidden_states=True)
    last_hidden = embedding['hidden_states'][-1].detach().numpy()
    return last_hidden.reshape(last_hidden.shape[0], last_hidden.shape[1] * last_hidden.shape[2])

## General Setup

In [6]:
# Load Data
path = kagglehub.dataset_download('abhi8923shriv/sentiment-analysis-dataset')
train_dataset = path+'/train.csv'
test_dataset = path+'/test.csv'
train_df = pd.read_csv(train_dataset, encoding='ISO-8859-1')
test_df = pd.read_csv(test_dataset, encoding='ISO-8859-1')

In [7]:
# Basic Preprocessing (need to update with preprocessor)
train = train_df.dropna(subset = "text")[["text", "sentiment"]]
test = test_df.dropna()[["text", "sentiment"]]

x = train["text"].values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train["sentiment"].values)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, stratify=y, random_state=42, test_size=0.3, shuffle=True)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=3)

## Using Embedding

In [8]:
#### Vectorise X by embedding model
x_train_transformed = embed_model(**tokenizer(x_train.tolist(), padding="max_length", max_length=60, truncation=True, return_tensors='pt').to(device))
x_valid_transformed =  embed_model(**tokenizer(x_valid.tolist(), padding="max_length", max_length=60, truncation=True, return_tensors='pt').to(device))

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4727439360 bytes.

In [ ]:
print(x_train_transformed.shape, x_valid_transformed.shape)

(100, 61440) (100, 61440)


In [ ]:
# base layers
input_layer = ("input", {"shape": (x_train_transformed.shape[1], )})
dense_256 = ("dense", {"units": 256, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
dropout = ("dropout", {}) 
dense_128 = ("dense", {"units": 128, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
dense_64 = ("dense", {"units": 64, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
output_layer = ("output", {"units": 3, "activation": "softmax"})

In [ ]:
l = [input_layer, dense_256, dropout, dense_64, dense_64, dropout, output_layer]
model = models.TfModel(l, batch_size=32, dropout_rate=0.4)

In [ ]:
model.fit(x_train_transformed, y_train)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.3575 - loss: 8.1112 - learning_rate: 1.0000e-04
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.5098 - loss: 7.8767 - learning_rate: 9.0484e-05
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.6011 - loss: 7.6941 - learning_rate: 8.1873e-05
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7378 - loss: 7.3543 - learning_rate: 7.4082e-05
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7265 - loss: 7.3643 - learning_rate: 6.7032e-05
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7818 - loss: 7.2393 - learning_rate: 6.0653e-05
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7130 - loss: 7.3290 - learning_rate: 5.4881e-05
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.8271 - loss: 7.2405 - learning_rate: 4.9659e-05
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.8549 - loss: 6.9233 - learning_rate: 4.4

TfModel(batch_size=32, dropout_rate=0.4,
        layer_list=[('input', {'shape': (61440,)}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CEB0F8750>,
                      'units': 256}),
                    ('dropout', {}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CF114B510>,
                      'units': 64}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CF114B510>,
                      'units': 64}),
                    ('dropout', {}),
                    ('output', {'activation': 'softmax', 'units': 3})])

In [ ]:
preds = model.predict(x_valid_transformed)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

           0       0.52      0.44      0.48        36
           1       0.49      0.62      0.55        34
           2       0.58      0.50      0.54        30

   micro avg       0.52      0.52      0.52       100
   macro avg       0.53      0.52      0.52       100
weighted avg       0.52      0.52      0.52       100
 samples avg       0.52      0.52      0.52       100



## Using last hidden state

In [ ]:
#### Vectorise X by embedding model
x_train_transformed = get_last_hidden_state(x_train[:100].tolist())
x_valid_transformed =  get_last_hidden_state(x_valid[:100].tolist())

In [ ]:
print(x_train_transformed.shape, x_valid_transformed.shape)

(100, 61440) (100, 61440)


In [ ]:
# base layers
input_layer = ("input", {"shape": (x_train_transformed.shape[1], )})
dense_256 = ("dense", {"units": 256, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
dropout = ("dropout", {}) 
dense_128 = ("dense", {"units": 128, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
dense_64 = ("dense", {"units": 64, "kernel_regularizer": tf.keras.regularizers.l2(0.01), "activation":"relu"})
output_layer = ("output", {"units": 3, "activation": "softmax"})

In [ ]:
l = [input_layer, dense_256, dropout, dense_64, dense_64, dropout, output_layer]
model = models.TfModel(l, batch_size=32, dropout_rate=0.4)

In [ ]:
model.fit(x_train_transformed, y_train[:100])

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.3575 - loss: 8.1112 - learning_rate: 1.0000e-04
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.5098 - loss: 7.8767 - learning_rate: 9.0484e-05
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.6011 - loss: 7.6941 - learning_rate: 8.1873e-05
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7378 - loss: 7.3543 - learning_rate: 7.4082e-05
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7265 - loss: 7.3643 - learning_rate: 6.7032e-05
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7818 - loss: 7.2393 - learning_rate: 6.0653e-05
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7130 - loss: 7.3290 - learning_rate: 5.4881e-05
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.8271 - loss: 7.2405 - learning_rate: 4.9659e-05
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.8549 - loss: 6.9233 - learning_rate: 4.4

TfModel(batch_size=32, dropout_rate=0.4,
        layer_list=[('input', {'shape': (61440,)}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CEB0F8750>,
                      'units': 256}),
                    ('dropout', {}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CF114B510>,
                      'units': 64}),
                    ('dense',
                     {'activation': 'relu',
                      'kernel_regularizer': <keras.src.regularizers.regularizers.L2 object at 0x0000019CF114B510>,
                      'units': 64}),
                    ('dropout', {}),
                    ('output', {'activation': 'softmax', 'units': 3})])

In [ ]:
preds = model.predict(x_valid_transformed)
print(classification_report(y_valid[:100], preds))

              precision    recall  f1-score   support

           0       0.52      0.44      0.48        36
           1       0.49      0.62      0.55        34
           2       0.58      0.50      0.54        30

   micro avg       0.52      0.52      0.52       100
   macro avg       0.53      0.52      0.52       100
weighted avg       0.52      0.52      0.52       100
 samples avg       0.52      0.52      0.52       100

